<img src="segmind-logo.png" style="width:350px;height:80px"/>

<h1><center>Welcome to SegMind 🙌</center></h1>

<img src="ml_meme.png" style="width:650px;height:400px"/>

#### In this Jupyter notebook we will learn about how to use Segmind platform 👩‍💻

## 1. How to get started 🚦

         1. Go to + New on the top right 
         2. New Notebook 📘
         3. Select the environment, machine size, storage
         4. Click start 🏁
         
         Off you go! 🏎

<img src="segmind_new_nb.gif" style="width:900px;height:500px"/>

## 2. Change instances 🤹‍♂️

         1. Click on the settings on the top right ⚙️
         2. Go to settings
         3. Change instance
         4. Select the size
         5. Click start 🏁
         6. Wait for the status to turn to "running" 🟢🏃‍♂️
         
         Viola! You're on a new machine in a jiffy 💪💪

<img src="segmind_change_nb.gif" style="width:900px;height:500px"/>

## Let's take a look at an example - Fashion MNIST 

<img src="fashion-mnist-sprite.png" style="width:650px;height:400px"/>

<img src="embedding.gif" style="width:650px;height:400px"/>

Labels

Each training and test example is assigned to one of the following labels:

Label	Description

    0	T-shirt/top 👚
    1	Trouser 👖
    2	Pullover 🥼
    3	Dress 👗
    4	Coat 🧥 
    5	Sandal 🩴
    6	Shirt 👕
    7	Sneaker 👟
    8	Bag 🎒
    9	Ankle boot 🥾

### 1. Import libraries

Below we import some segmind libraries for automatic experiment tracking

In [ ]:
from tensorflow import keras
from segmind import log_params_decorator, log_param
from segmind.keras import KerasCallback
from segmind import __version__
print(__version__)

### 2. Import the fashion MNIST dataset from the keras library

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

### 3. Next we define the network for training

In [ ]:
@log_params_decorator ## log_params_decorator automatically records and tracks the parameters of your network
def define_mnist_model(input_shape, hidden_neurons, num_classes=10):
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)),
        keras.layers.Dense(hidden_neurons, activation='relu'),
        keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

### 4. Normalize the images

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

### 5. Declare the model

In [ ]:
model = define_mnist_model((28, 28), hidden_neurons=28)

### 6. Initiate hyperparameters

In [ ]:
initial_lr=0.001
log_param(key='initial_leraning_rate', value=initial_lr)
lr = keras.optimizers.Adam(learning_rate=initial_lr)

### 7. Compile the model

In [ ]:
model.compile(
    optimizer=lr,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])
keras_cb = KerasCallback()

### 8. Start training

In [ ]:
model.fit(
    train_images,
    train_labels,
    validation_data=(test_images, test_labels),
    validation_steps=10,
    validation_freq=1,
    epochs=20,
    steps_per_epoch=10,
    callbacks=[keras_cb])